In [ ]:
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.mlab import griddata
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
class Problema(object):
    
    estado_inicial = None
    
    def vizinhos(self, estado):
        raise NotImplementedError
    
    def sol_otima(self):
        return None

class BuscaLocal(object):
    def __init__(self):
        self.estados = []
    
    def buscar(self, problema):
        raise NotImplementedError
        
    def log_estado(self, estado):
        self.estados.append(estado)
        
    def plot_valores(self):
        plt.figure()
        valores = map(lambda x: x.valor, self.estados)
        plt.plot(np.arange(len(self.estados)), valores)
        plt.ylabel('Valor')
        plt.xlabel('Iteracao')
        plt.show()

class Estado(object):
    def __init__(self, valor):
        self.valor = valor

In [ ]:
class EstadoPSVM(Estado):
    def __init__(self, valor, e, d):
        super(EstadoPSVM, self).__init__(valor)
        self.e = e
        self.d = d
    
    def __str__(self):
        return 'E=%d, D=%d, S=%s' % (self.e, self.d, self.valor)
        
class ProblemaSubVetorMaximo(Problema):
    
    @classmethod
    def from_aleatorio(cls, n_elementos, e_inicial=None, d_inicial=None, r_seed=None):
        rs = np.random.RandomState(r_seed)
        
        cls.vetor = rs.randint(-100,100,(n_elementos))
        
        if not e_inicial:
            e = rs.randint(0, n_elementos-1)
            d = rs.randint(e, n_elementos)
        else:
            e = e_inicial
            d = d_inicial
        valor_ini = ProblemaSubVetorMaximo.__valor_solucao(cls.vetor,e,d)
        cls.estado_inicial = EstadoPSVM(valor_ini, e, d)
        return cls()
    
    @classmethod
    def from_vetor(cls, vetor, e_inicial, d_inicial):
        cls.vetor = vetor
        cls.estado_inicial = EstadoPSVM(ProblemaSubVetorMaximo.__valor_solucao(vetor,e_inicial, d_inicial), e_inicial, d_inicial)
        return cls()
        
    @staticmethod
    def __valor_solucao(v, e, d):
        return np.sum(v[e:d+1])
    
    def valor_solucao(self, e, d):
        return ProblemaSubVetorMaximo.__valor_solucao(self.vetor, e, d)
    
    def vizinhos(self, estado):
        vizs = []
        e = estado.e
        d = estado.d
        for i in [e-1,e,e+1]:
            if (i < 0) or (i >= self.vetor.shape[0]):
                continue
            for j in [d-1,d,d+1]:
                if (j < 0) or (j >= self.vetor.shape[0]):
                    continue
                if (e == i) and (d == j):
                    continue
                vizs.append(EstadoPSVM(self.valor_solucao(i,j), i, j))
        return vizs
    
    def sol_otima(self):
        #Find maximum subarray (Cormen)
        def fmsa(V, e, d):
            #print (e, d)
            if e == d:
                return (e, d, V[e])
            else:
                meio = int(math.floor((e + d) / 2))
                
                eb, ea, es = fmsa(V, e, meio)
                db, da, ds = fmsa(V, meio + 1, d)
                cb, ca, cs = fmcsa(V, e, meio, d)
                if (es >= ds) and (es >= cs):
                    return (eb, ea, es)
                elif (ds >= es) and (ds > cs):
                    return (db, da, ds)
                else:
                    return (cb, ca, cs)
        
        def fmcsa(V, e, m, d):
            es = -np.infty
            s = 0
            for i in range(m, e-1, -1):
                s += V[i]
                if s > es:
                    es = s
                    em = i
            ds = -np.infty
            s = 0
            for j in range(m+1, d+1):
                s = s + V[j]
                if s > ds:
                    ds = s
                    dm = j
            return (em, dm, es + ds)
                    
        solucao = fmsa(self.vetor, 0, self.vetor.shape[0]-1)
            
        return EstadoPSVM(solucao[2], solucao[0], solucao[1])
    
    def plot_espaco_estados(self, estados):
        X = np.array(map(lambda x: x.e, estados))
        Y = np.array(map(lambda x: x.d, estados))
        Z = np.array(map(lambda x: x.valor, estados))
        
        xi = np.linspace(min(X), max(X))
        yi = np.linspace(min(Y), max(Y))
        
        xx, yy = np.meshgrid(xi, yi)
        zz = griddata(X, Y, Z, xi, yi, interp='linear')
        
        plt.pcolormesh(xx,yy,zz, cmap=cm.jet)
        plt.show()


In [ ]:
class HillClimbingBL(BuscaLocal):
    
    def __init__(self, tolerancia=None):
        super(HillClimbingBL, self).__init__()        
        self.tolerancia = tolerancia
    
    def buscar(self, problema):
        atual = problema.estado_inicial
        it = 0
        estag = 0
        melhor = None
        while True:
            print('###Iteração %d###' % it)
            vzs = problema.vizinhos(atual)
            print_vizinhos(vzs)
            vizinho = max(vzs, key=lambda x: x.valor)
            print('Vizinho escolhido: %s' % vizinho)
            
            if not self.tolerancia:
                if vizinho.valor <= atual.valor:
                    return atual
            atual = vizinho
            
            if melhor is not None:
                if atual.valor > melhor.valor:
                    melhor = atual
                    estag = 0
                else:
                    estag +=1
                    if estag > self.tolerancia:
                        return melhor
            else:
                melhor = atual
            
            self.log_estado(atual)
            
            print('Estado atual: %s' % atual)
            if self.tolerancia:
                print('Melhor estado: %s' % melhor)
            print('')
            it+=1
            

def print_vizinhos(estados):
    print('Vizinhos')
    for e in estados:
        print('\t%s' % (e))
            

In [ ]:
#p = ProblemaSubVetorMaximo.from_aleatorio(10)

#v = np.array([13, -3, -25, 20, -3, -16, -23, 18, 20, -7, 12, -5, -22, 15, -4, 7])
#p = ProblemaSubVetorMaximo.from_vetor(v, 5, 12)

p = ProblemaSubVetorMaximo.from_aleatorio(50, e_inicial=20, d_inicial=24, r_seed=50)

hc = HillClimbingBL(tolerancia=10)

print(p.vetor)
print ('Estado Inicial: %s' % p.estado_inicial)

solucao = hc.buscar(p)

print("Solução: %s" % solucao)

sol_otima = p.sol_otima()
print ('Solução ótima: %s' % sol_otima)

hc.plot_valores()

p.plot_espaco_estados(hc.estados)
